<a href="https://www.kaggle.com/code/mh0386/medical-insurance-cost-prediction?scriptVersionId=120919838" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd

# The Data

In [ ]:
data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
data